## Data set 

In [ ]:
import pandas as pd

In [ ]:
churn=pd.read_csv('train_churn_kg.csv')

## Data exploration 

In [ ]:
churn.head(10)

In [ ]:
churn.shape

In [ ]:
churn.info()

In [ ]:
churn.describe()

## data cleaning

In [ ]:
churn_updated=churn.iloc[:,2:]

In [ ]:
churn_updated.head(10)

In [ ]:
churn_updated.isnull().sum()

In [ ]:
churn_updated['CHURN'].value_counts()

In [ ]:
churn_updated.dropna(axis=0,inplace=True)

In [ ]:
churn_updated.isnull().sum()

In [ ]:
churn_updated.shape

In [ ]:
#def greater(var):
   # for i in range(0,len(var)):
       # if churn_updated[var[i]]<0:
           # churn_updated.drop(churn_updated[var[i]])
#var=['DAYS_LIFE','DEVICE_TECNOLOGY','MIN_PLAN','PRICE_PLAN','TOT_MIN_CALL_OUT','AVG_MIN_CALL_OUT_3','TOT_MIN_IN_ULT_MES','AVG_MIN_IN_3','ROA_LASTMONTH','ROACETEL_LAST_MONTH','DEVICE','TEC_ANT_DATA','STATE_DATA','CITY_DATA','TEC_ANT_VOICE','STATE_VOICE','CITY_VOICE','CHURN']
#greater(var)
#print(len(var))

In [ ]:
churn_updated[(churn_updated['DAYS_LIFE']<0) | (churn_updated['STATE_DATA']<0) | (churn_updated['CITY_DATA']<0) |(churn_updated['CITY_VOICE']<0) ]

In [ ]:
c_updated = churn_updated.drop(churn_updated[(churn_updated['DAYS_LIFE']<0) | (churn_updated['STATE_DATA']<0) | (churn_updated['CITY_DATA']<0) |(churn_updated['CITY_VOICE']<0) ].index)

In [ ]:
c_updated.shape

In [ ]:
c_updated[(c_updated['DAYS_LIFE']<0) | (c_updated['STATE_DATA']<0) | (c_updated['CITY_DATA']<0) |(c_updated['CITY_VOICE']<0) ]

## Feature Engineering 

In [ ]:
import seaborn as sns

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
c_updated.drop(['CHURN'],axis=1).corr()

In [ ]:
corr=c_updated.drop(['CHURN'],axis=1).corr()
plt.figure(figsize=(20,18))
sns.heatmap(corr,annot=True,cmap='RdYlGn_r')

In [ ]:
col=corr.columns
print(col)

In [ ]:
columns=[c for c in corr]
co_col={}
for y in corr.iterrows():
    #print(len(y[1]))
    for i in range(len(y[1])):        
        if y[1][i]>0.8:
            if y[0]!=columns[i]:
                #print(y[1][i],columns[i],y[0])
                co_col[columns[i]]=y[1][i]
print(co_col)
    

In [ ]:
cols_to_drop=['TOT_MIN_CALL_OUT','STATE_DATA','CITY_DATA']

In [ ]:
c_updtd=c_updated.drop(cols_to_drop,axis=1)

In [ ]:
c_updtd.head()

In [ ]:
x=c_updtd.drop(['CHURN'],axis=1)
y=c_updtd['CHURN']

## Model Building 

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=10,test_size=0.3,stratify=y)

In [ ]:
print(x_train.shape)
print(y_train.shape)


## Hyper parameter Tuning 

In [ ]:
from sklearn.model_selection import GridSearchCV


In [ ]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5,10]
        }
    },
    'decision':
            {
                 'model':DecisionTreeClassifier(),
                 'parameters':
                             {
                                 'criterion':['gini','entropy'],
                                 'splitter':'best'
                             }
             }
}

In [ ]:

scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(x_train,y_train)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df.head()